# Installments

In [ ]:
#https://github.com/beir-cellar/beir/blob/main/examples/retrieval/evaluation/dense/evaluate_dpr.py
!pip install beir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 681.2 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.2/219.2 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.4 MB/s eta 0:00:00
  Created wheel for beir: filename=beir-2.0.0-py3-none-any.whl size=63550 sha256=de61f8410e25bbfdd4d690fe15ceacddc3818da562837937ae4ab1539cb68e19
  Stored in directory: /root/.cache/pip/wheels/1c/14/96/c606ede3c10e9300ef771a

# Import Libraries

In [ ]:
import numpy as np
import pandas as pd

from time import time
from beir import util, LoggingHandler
from beir.retrieval import models
from beir.datasets.data_loader import GenericDataLoader
from beir.retrieval.evaluation import EvaluateRetrieval
from beir.retrieval.search.dense import DenseRetrievalExactSearch as DRES
from sentence_transformers import losses, models, SentenceTransformer
from beir.retrieval.train import TrainRetriever


import logging
import pathlib, os
import random

## Connect To Google Drive And Load Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls '/content/drive/MyDrive/project_data'

scifact


In [ ]:
!ls '/content/drive/MyDrive/project_data/scifact'

corpus.jsonl	    queries.jsonl  test.source	   train.csv	    train.target     val.source_rl
process_scifact.py  rephrase	   test.source_rl  train.source     train.target_rl  val.target
qrels		    test.csv	   test.target	   train.source_rl  val.source	     val.target_rl


In [ ]:
data_directory = '/content/drive/MyDrive/project_data'

# Load Meta-Data

In [ ]:
corpus = pd.read_json(data_directory + '/scifact/corpus.jsonl', lines=True, dtype=str)
corpus

,_id,title,text,metadata
0,4983,Microstructural development of human newborn c...,Alterations of the architecture of cerebral wh...,{}
1,5836,Induction of myelodysplasia by myeloid-derived...,Myelodysplastic syndromes (MDS) are age-depend...,{}
2,7912,"BC1 RNA, the transcript from a master gene for...",ID elements are short interspersed elements (S...,{}
3,18670,The DNA Methylome of Human Peripheral Blood Mo...,DNA methylation plays an important role in bio...,{}
4,19238,The human myelin basic protein gene is include...,Two human Golli (for gene expressed in the oli...,{}
...,...,...,...,...
5178,195689316,Body-mass index and cause-specific mortality i...,BACKGROUND The main associations of body-mass ...,{}
5179,195689757,Targeting metabolic remodeling in glioblastoma...,A key aberrant biological difference between t...,{}
5180,196664003,Signaling architectures that transmit unidirec...,A signaling pathway transmits information from...,{}
5181,198133135,"Association between pre-diabetes, type 2 diabe...",AIMS Trabecular bone score (TBS) is a surrogat...,{}


# Load MyQueries and qrels from .csv files

In [ ]:
#train data
df_train = pd.read_csv(data_directory + '/scifact/train.csv', sep='\t', dtype=str)
train_q = df_train[['qid', 'query']]

#test_data
df_test = pd.read_csv(data_directory + '/scifact/test.csv', sep='\t', dtype=str)
test_q = df_test[['qid', 'query']]

test_q

,qid,query
0,1,0-dimensional biomaterials show inductive prop...
1,3,"1,000 genomes project enables mapping of genet..."
2,5,1/2000 in UK have abnormal PrP positivity.
3,13,5% of perinatal mortality is due to low birth ...
4,36,A deficiency of vitamin B12 increases blood le...
...,...,...
334,1379,Women with a higher birth weight are more like...
335,1382,aPKCz causes tumour enhancement by affecting g...
336,1385,cSMAC formation enhances weak ligand signalling.
337,1389,mTORC2 regulates intracellular cysteine levels...


In [ ]:
train_q1 = pd.read_csv(data_directory + '/scifact/rephrase/my_train_query1.csv', sep='\t', index_col=0, dtype=str)
train_q2 = pd.read_csv(data_directory + '/scifact/rephrase/my_train_query2.csv', sep='\t', index_col=0, dtype=str)
train_q3 = pd.read_csv(data_directory + '/scifact/rephrase/my_train_query3.csv', sep='\t', index_col=0, dtype=str)
train_q4 = pd.read_csv(data_directory + '/scifact/rephrase/my_train_query4.csv', sep='\t', index_col=0, dtype=str)
train_q5 = pd.read_csv(data_directory + '/scifact/rephrase/my_train_query5.csv', sep='\t', index_col=0, dtype=str)
train_q6 = pd.read_csv(data_directory + '/scifact/rephrase/my_train_query6.csv', sep='\t', index_col=0, dtype=str)

,qid,query
0,1,0 dimensional biomaterials show inductive prop...
1,3,1 000 genomes project enables mapping of genet...
2,5,1 2000 in uk have abnormal prp positivity 1 20...
3,13,5 of perinatal mortality is due to low birth w...
4,36,a deficiency of vitamin b12 increases blood le...
...,...,...
334,1379,women with a higher birth weight are more like...
335,1382,apkcz causes tumour enhancement by affecting g...
336,1385,csmac formation enhances weak ligand signallin...
337,1389,mtorc2 regulates intracellular cysteine levels...


In [ ]:
#train qrels
df_train_qrels = pd.read_csv(data_directory + '/scifact/qrels/train.tsv', sep='\t', dtype=str)

#test qrels
df_test_qrels = pd.read_csv(data_directory + '/scifact/qrels/test.tsv', sep='\t', dtype=str)

df_test_qrels

,query-id,corpus-id,score
0,1,31715818,1
1,3,14717500,1
2,5,13734012,1
3,13,1606628,1
4,36,5152028,1
...,...,...,...
334,1379,17450673,1
335,1382,17755060,1
336,1385,306006,1
337,1389,23895668,1


# Rewriting data-format to dict

In [ ]:
#Note that _id here refers to doc id.
new_corpus = {}
for i in range(len(corpus)):
    key = str(corpus['_id'][i])
    new_corpus[key] = {'text' : corpus['text'][i], 'title' : corpus['title'][i]}

In [ ]:
def dict_query(data):
    new_dict = {}
    for i in range(len(data)):
      key = str(data['qid'][i])
      new_dict[key] = data['query'][i]

    return new_dict

In [ ]:
dict_tr_q = dict_query(train_q)
dict_tr_q1 = dict_query(train_q1)
dict_tr_q2 = dict_query(train_q2)
dict_tr_q3 = dict_query(train_q3)
dict_tr_q4 = dict_query(train_q4)
dict_tr_q5 = dict_query(train_q5)
dict_tr_q6 = dict_query(train_q6)

dict_te_q = dict_query(test_q)

In [ ]:
#train_qrels
new_train_qrels = {}
for i in range(len(df_train_qrels)):
    query_key = str(df_train_qrels['query-id'][i])
    new_train_qrels[query_key] = {str(df_train_qrels['corpus-id'][i]) : int(df_train_qrels['score'][i])}

#test_qrels
new_test_qrels = {}
for i in range(len(df_test_qrels)):
    query_key = str(df_test_qrels['query-id'][i])
    new_test_qrels[query_key] = {str(df_test_qrels['corpus-id'][i]) : int(df_test_qrels['score'][i])}

# Training - Supervised DPR

In [ ]:
model_name = "msmarco-distilbert-base-v3"
model_norm = SentenceTransformer(model_name)
model_comb = SentenceTransformer(model_name)


retriever_norm = TrainRetriever(model=model_norm, batch_size=16)
retriever_comb = TrainRetriever(model=model_comb, batch_size=16)

train_samples_norm = retriever_norm.load_train(new_corpus, dict_tr_q, new_train_qrels)
train_samples_comb = retriever_comb.load_train(new_corpus, dict_tr_q, new_train_qrels)

train_dataloader_norm = retriever_norm.prepare_train(train_samples_norm, shuffle=True)
train_dataloader_comb = retriever_comb.prepare_train(train_samples_comb, shuffle=True)

#### Training SBERT with cosine-product
train_loss_norm = losses.MultipleNegativesRankingLoss(model=retriever_norm.model)
train_loss_comb = losses.MultipleNegativesRankingLoss(model=retriever_comb.model)
#### training SBERT with dot-product
#train_loss_norm = losses.MultipleNegativesRankingLoss(model=retriever_norm.model, similarity_fct=util.dot_score)
#train_loss_comb = losses.MultipleNegativesRankingLoss(model=retriever_comb.model, similarity_fct=util.dot_score)

Adding Input Examples:   0%|          | 0/51 [00:00<?, ?it/s]

Adding Input Examples:   0%|          | 0/51 [00:00<?, ?it/s]

In [ ]:
#### Prepare dev evaluator
#ir_evaluator = retriever.load_ir_evaluator(dev_corpus, dev_queries, dev_qrels)

#### If no dev set is present from above use dummy evaluator
ir_evaluator_norm = retriever_norm.load_dummy_evaluator()
ir_evaluator_comb = retriever_comb.load_dummy_evaluator()

In [ ]:
#### Provide model save path
model_save_path_norm = os.path.join(pathlib.Path(data_directory + '/dpr_models').parent.absolute(), "output_norm", "{}-v1-{}".format(model_name, "SCI-FACT"))
os.makedirs(model_save_path_norm, exist_ok=True)

model_save_path_comb = os.path.join(pathlib.Path(data_directory + '/dpr_models').parent.absolute(), "output_comb", "{}-v1-{}".format(model_name, "SCI-FACT"))
os.makedirs(model_save_path_comb, exist_ok=True)

In [ ]:
#### Configure Train params
num_epochs = 5
evaluation_steps = 5000
warmup_steps_norm = int(len(train_samples_norm) * num_epochs / retriever_norm.batch_size * 0.1)
warmup_steps_comb = int(len(train_samples_comb) * num_epochs / retriever_comb.batch_size * 0.1)

In [ ]:
retriever_norm.fit(train_objectives=[(train_dataloader_norm, train_loss_norm)],
                evaluator=ir_evaluator_norm,
                epochs=num_epochs,
                output_path=model_save_path_norm,
                warmup_steps=warmup_steps_norm,
                evaluation_steps=evaluation_steps,
                use_amp=True)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

In [ ]:
retriever_comb.fit(train_objectives=[(train_dataloader_comb, train_loss_comb)],
                evaluator=ir_evaluator_comb,
                epochs=num_epochs,
                output_path=model_save_path_comb,
                warmup_steps=warmup_steps_comb,
                evaluation_steps=evaluation_steps,
                use_amp=True)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

# Evaluation - Supervised DPR

In [ ]:
from beir.retrieval import models

In [ ]:
def eval_train_model(model_path, corpus, test_data, qrels):
  evaluator_model = DRES(models.SentenceBERT(model_path, batch_size=16))
  evaluator = EvaluateRetrieval(evaluator_model, score_function="dot")
  results = evaluator.retrieve(corpus, test_data)
  ndcg, _map, recall, precision = evaluator.evaluate(qrels, results, [10,100,1000])
  return ndcg, _map, recall, precision

In [ ]:
norm = eval_train_model(model_save_path_norm, new_corpus, dict_te_q, new_test_qrels)

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/41 [00:00<?, ?it/s]

In [ ]:
comb = eval_train_model(model_save_path_comb, new_corpus, dict_te_q, new_test_qrels)

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/41 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/41 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/41 [00:00<?, ?it/s]

In [ ]:
print(norm)
print(comb)

({'NDCG@10': 0.57892, 'NDCG@100': 0.61138, 'NDCG@1000': 0.61804}, {'MAP@10': 0.51668, 'MAP@100': 0.52317, 'MAP@1000': 0.52341}, {'Recall@10': 0.77667, 'Recall@100': 0.93, 'Recall@1000': 0.98333}, {'P@10': 0.07767, 'P@100': 0.0093, 'P@1000': 0.00098})
({'NDCG@10': 0.47162, 'NDCG@100': 0.51855, 'NDCG@1000': 0.53143}, {'MAP@10': 0.4195, 'MAP@100': 0.42885, 'MAP@1000': 0.42929}, {'Recall@10': 0.64, 'Recall@100': 0.86333, 'Recall@1000': 0.96667}, {'P@10': 0.064, 'P@100': 0.00863, 'P@1000': 0.00097})
({'NDCG@10': 0.56759, 'NDCG@100': 0.6034, 'NDCG@1000': 0.60909}, {'MAP@10': 0.50576, 'MAP@100': 0.51325, 'MAP@1000': 0.51343}, {'Recall@10': 0.76333, 'Recall@100': 0.93, 'Recall@1000': 0.97667}, {'P@10': 0.07633, 'P@100': 0.0093, 'P@1000': 0.00098})
({'NDCG@10': 0.4809, 'NDCG@100': 0.52041, 'NDCG@1000': 0.53421}, {'MAP@10': 0.42498, 'MAP@100': 0.43179, 'MAP@1000': 0.43235}, {'Recall@10': 0.65667, 'Recall@100': 0.85667, 'Recall@1000': 0.96333}, {'P@10': 0.06567, 'P@100': 0.00857, 'P@1000': 0.0009